In [505]:
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np
import math


class load_whole_data(Dataset): # Dataset
    """
    This dataloader loads the tensor input and target in whole
    """
    def __init__(self, path: str, ind: list):
        """
        Args:
            path (str): path to the input & target folder.
            ind (list): list of indices for which pictures to load.
            device (class 'torch.device'): which pytorch device the data should
            be sent to.
        """

        self.device = "cpu"
        self.imgs_path = path
        self.data = []
        for i in ind:
            self.data.append([self.imgs_path + f"/model_input ({i}).pt",
                        self.imgs_path + f"/model_target ({i}).pt"])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_path, target_path = self.data[idx] # path for target + input

        inp = torch.load(input_path) # load the input data
        inp = inp.type(torch.float).to(self.device)

        tar = torch.load(target_path) # load the target data
        tar = tar.type(torch.float).to(self.device)

        return inp, tar
    


class load_5_min_intervals(Dataset):
    """
    This dataloader loads the tensor input and target in whole
    """
    def __init__(self, ls: list, device):
        """
        Args:
            path (str): path to the input & target folder.
            ind (list): list of indices for which pictures to load.
            device (class 'torch.device'): which pytorch device the data should
            be sent to.
        """

        self.device = device
        self.ls = ls # list with the input and target data
        self.size = [ls[0][0].shape[0], ls[0][0].shape[1]]
            # size of target and input
        
        # zero-pad the result if it can't be in only 5 mins intervals.
        extra = 250*60*5 - (w := (ls[0][0].shape[1]-30*250) % (250*60*5))
        
        
        if w: # if w is not equal to 0, then zero-pad is needed:
            # zero pad:
            self.ls[0] = F.pad(self.ls[0], (0, extra), "constant", 0.0)
            self.ls[1] = F.pad(self.ls[1], (0, extra), "constant", 0.0)

            self.size[1] = self.size[1] + extra
        

        self.length = math.floor((self.size[1]-30*250)/(250*60*5))*self.size[0]
            # the amount of total possible cuts

        self.gen = iter(self.cut_data())
        
        

    def cut_data(self):
        for chan in range(self.size[1]):
            for cut_point in range(30*250, self.size[1], 250*5*60):
                inp = self.ls[0][0][chan][cut_point:cut_point+60*5*250]
                
                tar = self.ls[1][0][chan][cut_point:cut_point+60*5*250]
                yield (inp, tar, chan, cut_point)

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        inp, tar, chan, cut = next(self.gen)
        inp = inp.to(self.device)
        tar = tar.to(self.device)
        return inp, tar, chan, cut

In [506]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

trainload = load_whole_data(path = "C:/Users/Marc/Desktop/model_data", ind = [i for i in range(1, 283 + 1)])
batch_size = 1

# Set up the dataloaders:

trainloader = torch.utils.data.DataLoader(trainload,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=0)
load = iter(trainloader)




cuda


In [513]:
loader2 = load_5_min_intervals(next(load), "cpu")
batch_size = 1

# Set up the dataloaders:

loader2 = torch.utils.data.DataLoader(loader2,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=0)
j = 0
for i in loader2:
    j += 1

print(i)
    

print(j)

[tensor([[3.0992, 2.7187, 2.4399,  ..., 0.0000, 0.0000, 0.0000]]), tensor([[0., 0., 0.,  ..., 0., 0., 0.]]), tensor([21]), tensor([1957500])]
594


In [515]:
594/22

27.0

In [516]:
5*27

135

In [496]:
def gener():
    for i in range(20):
        for j in range(4):
            yield (i, j)

In [409]:
d = iter(gener())

In [472]:
next(d)

(15, 2)

# 

In [317]:
[i for i in range(0, 301000 + 74000, 5*250*60)]

[0, 75000, 150000, 225000, 300000]

In [254]:
yet = 301000

In [255]:


extra = 250*60*5 - (w := yet % (250*60*5))



print(extra)
print(w)

74000
1000


In [256]:
(301000 + 74000) %(250*60*5)

0

In [306]:
250*60*5 - (w := (yet % (250*60*5)))

74000

In [305]:
k = v[0]
k.shape

torch.Size([1, 22, 301000])

In [304]:
t4d = torch.rand(5)
p1d = (2, 2)
out = F.pad(t4d, p1d, "constant", 0.0)  # effectively zero padding
print(out.size())
print(out)

torch.Size([9])
tensor([0.0000, 0.0000, 0.9434, 0.6756, 0.5096, 0.1459, 0.4407, 0.0000, 0.0000])


In [297]:
p1d = (0, 74000) # pad last dim by 1 on each side
out = F.pad(k, p1d, "constant", 0.0)  # effectively zero padding
print(out.size())

torch.Size([1, 22, 375000])


tensor([0.2925, 0.8081, 0.0816, 0.9448, 0.2459])

In [294]:
74000/2

37000.0